<a href="https://colab.research.google.com/github/intrinio/intrinio-tutorials/blob/master/standardized_fundamentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This script is intended to show the finer points of pulling standardized fundamentals from the Intrinio API. It shows how to pull the statement IDs for a given ticker, filtering by balance sheet. Then, it takes those statement IDs and uses them to return a dataframe of the statement. 

This script assumes that you already have an API key and that it is stored as an environment variable using colab-env. 

If you don't have an API key, head to https://www.intrinio.com. 

If you are unfamilar with storing your API keys as an environment variable here is a tutorial: https://github.com/intrinio/intrinio-tutorials/blob/master/environment_variable_set_up.ipynb


In [ ]:
#Installing colab-env and intrinio_sdk

!pip install colab-env -qU
!pip install intrinio_sdk -qU

#Require packages we will need for this particular script
from __future__ import print_function
import colab_env
import pandas as pd
from psycopg2 import connect
import os
import intrinio_sdk
from intrinio_sdk.rest import ApiException
from pprint import pprint

#Setting option to print 2 decimals so that we can make cents on the filings
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [2]:
#Call your intrinio api key from your GDrive Passwords folder
intrinio_api_key = os.getenv("intrinio_api_key")


The [Intrinio documentation](https://docs.intrinio.com/documentation/python/get_fundamental_standardized_financials_v2) for standardized financials makes it clear that you can use the Intrinio ID, which is a unique identifier for a financial statement, or the lookup code, formatted like this: 

```
id = 'AAPL-income_statement-2018-Q1'```

This ID will pull in Apple's income statement for Q1 in 2018. This can be rather inefficient, especially if you want to pull all of Apple's statements or if you don't know when the most recent statement was released.

A more efficient method is to pull a list of Apple's financial statements and then to use the unique IDs to pull the most recent statement, or all statements. You can see the lookup [documentation here](https://docs.intrinio.com/documentation/python/lookup_company_fundamental_v2).

In [3]:
#First, open a connection to the Intrinio API

#Connect to Intrinio API  
intrinio_sdk.ApiClient().configuration.api_key['api_key'] = intrinio_api_key

#We will use the fundamentals functionality
fundamentals_api = intrinio_sdk.FundamentalsApi()

#And the company functionality 
company_api = intrinio_sdk.CompanyApi()

In [4]:
#Setting an example ticker of Apple
ticker = 'AAPL'

#Using the company function to return only balance sheet statements, both standardized and as reported, for that ticker
intrinio_filings = company_api.get_company_fundamentals(ticker, reported_only = False, statement_code = 'balance_sheet_statement')

#From the array of responses, selecting the most recent ID. This is the unique identifier for Apple's most recent balance sheet statement.  
id = intrinio_filings.fundamentals_dict[0].get('id')



In [5]:
#Now, using the id we returned above, we try and get an api response. If it fails, print oops!

try:
  api_response = fundamentals_api.get_fundamental_standardized_financials(id)
except: 
  print('oops!')

In [ ]:
#The api response has many parts- meta data, data about the api call. What we want is a dataframe of the balance sheet.
#The loop below pulls the standardized financials out of the response and converts them to a dataframe. 

statement = {}
for standardized_financial in api_response.standardized_financials:
  statement[standardized_financial.data_tag.tag] = standardized_financial.value

#Convert to df
intrinio_balance_sheet = pd.DataFrame(statement, index = [0])

In [6]:
#Here it is! The most recent balance sheet in a dataframe from the ticker selected. 
intrinio_balance_sheet

,cashandequivalents,shortterminvestments,notereceivable,accountsreceivable,netinventory,othercurrentassets,totalcurrentassets,netppe,longterminvestments,othernoncurrentassets,totalnoncurrentassets,totalassets,shorttermdebt,accountspayable,currentdeferredrevenue,othercurrentliabilities,totalcurrentliabilities,longtermdebt,othernoncurrentliabilities,totalnoncurrentliabilities,totalliabilities,commitmentsandcontingencies,commonequity,retainedearnings,aoci,totalcommonequity,totalequity,totalequityandnoncontrollinginterests,totalliabilitiesandequity
0,40174000000.00,53877000000.00,14955000000.00,15722000000.00,3334000000.00,15691000000.00,143753000000.00,35889000000.00,98793000000.00,41965000000.00,140758000000.00,320400000000.00,20421000000.00,32421000000.00,5928000000.00,37324000000.00,96094000000.00,89086000000.00,56795000000.00,145881000000.00,241975000000.00,0.00,48032000000.00,33182000000.00,-2789000000.00,78425000000.00,78425000000.00,78425000000.00,320400000000.00


Now that you have the most recent balance sheet, you could write a loop and pull historical balance sheets to add to the dataframe. 

It is helpful to add the date to the response, shown below.

In [7]:
#Set index to be dates
intrinio_balance_sheet['filing_date'] = api_response.fundamental.end_date

intrinio_balance_sheet = intrinio_balance_sheet.set_index(['filing_date'])

In [8]:
intrinio_balance_sheet

,cashandequivalents,shortterminvestments,notereceivable,accountsreceivable,netinventory,othercurrentassets,totalcurrentassets,netppe,longterminvestments,othernoncurrentassets,totalnoncurrentassets,totalassets,shorttermdebt,accountspayable,currentdeferredrevenue,othercurrentliabilities,totalcurrentliabilities,longtermdebt,othernoncurrentliabilities,totalnoncurrentliabilities,totalliabilities,commitmentsandcontingencies,commonequity,retainedearnings,aoci,totalcommonequity,totalequity,totalequityandnoncontrollinginterests,totalliabilitiesandequity
filing_date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-03-28,40174000000.00,53877000000.00,14955000000.00,15722000000.00,3334000000.00,15691000000.00,143753000000.00,35889000000.00,98793000000.00,41965000000.00,140758000000.00,320400000000.00,20421000000.00,32421000000.00,5928000000.00,37324000000.00,96094000000.00,89086000000.00,56795000000.00,145881000000.00,241975000000.00,0.00,48032000000.00,33182000000.00,-2789000000.00,78425000000.00,78425000000.00,78425000000.00,320400000000.00
